## Basics of Convolution Neural Networks.

What are CNNs?
Well, CNN are type of neural network where in we convolute input/hidden layer with the weights. 
Weights in typical neural network is one-to-one mapped to each input/intermediate neurons. But,
in CNN, weights are shared between the patches of input/intermediate neurons.

Why share weights and what imapct does it have on neural networks?
Sharing weights solves two main problems </p> a) **translation invariance**; b) **Number of parameters to train in Networks**

### a) translation invariance:

Suppose you are training a neural network to detect a cat in a image. You really don't care where is the cat in the image, you want your model to be robust to detect cat irrespective of its position in the image, because a cat in right side of image is same as cat in left side of image.

If we train a traditional neural netwwork we may need to train every parameter(weights associated with each input pixel, teaching same thing to different neuron to detect the cat in different parts of image. And it fails to generalize.

This can be well handled by CNN were we share weights(patch/kernel) across the image. In CNN, parameters are shared acrossed space, similarly, in RNN it is shared accrossed time. Sharing weights help the cnn to detect already learnt object without retraining another set of parameters.

![parts of CNN](./images/CNN_01.png)

### b) Number of parameters to train in Networks

Number of parameters to train in neural networks is very important, because number of parameters is directly proportional to the time taken by nueral network to train. SO, while build a deep model(NN with many hidden layers) It is utmost important to know how big is NN in terms of parameters to trainig.


## Dimensionality

how can we calculate the number of neurons of each layer in our CNN?

Given:

our input layer has a width of W and a height of H
our convolutional layer has a filter size F
we have a stride of S
a padding of P
and the number of filters K,
the following formula gives us the width of the next layer: W_out = [(W−F+2P)/S] + 1.

The output height would be H_out = [(H-F+2P)/S] + 1.

And the output depth would be equal to the number of filters D_out = K.

The output volume would be W_out * H_out * D_out.

Knowing the dimensionality of each additional layer helps us understand how large our model is and how our decisions around filter size and stride affect the size of our network.



## Parameter sharing:


Let's see how to implement a Convnet in tensorflow.</p>
tensorflow provides the `tf.nn.conv2d()` and `tf.nn.bias_add()` function to create your own convolutional layers.

In [1]:
import tensorflow as tf


In [2]:
# Output depth
k_output = 64

# Image Properties
image_width = 10
image_height = 10
color_channels = 3

# Convolution filter
filter_size_width = 5
filter_size_height = 5

#Input/Image
input = tf.placeholder(tf.float32, shape=[None, image_height, image_width, color_channels])

#Weight and bias
weight = tf.Variable(tf.truncated_normal(shape=[filter_size_height,filter_size_width,color_channels, k_output]))
bias = tf.Variable(tf.zeros(k_output))

#Apply convolution
conv_layer = tf.nn.conv2d(input, weight, strides=[1,2,2,1], padding='SAME')
#Add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
#Apply activation function
conv_layer = tf.nn.relu(conv_layer)

The code above uses the `tf.nn.conv2d()` function to compute the convolution with weight as the filter and **[1, 2, 2, 1]** for the strides. TensorFlow uses a stride for each `input` dimension, `[batch, input_height, input_width, input_channels]`. We are generally always going to set the `stride` for `batch` and `input_channels` (i.e. the first and fourth element in the strides array) to be **1**.

You'll focus on changing `input_height` and `input_width` while setting `batch` and `input_channels` to **1**. The `input_height` and `input_width` strides are for striding the filter over `input`. This example code uses a stride of 2 with 5x5 filter over `input`.

The `tf.nn.bias_add()` function adds a 1-d bias to the last dimension in a matrix.

## Tensorflow Max Pooling

Conceptually, the benefit of the max pooling operation is to reduce the size of the input, and allow the neural network to focus on only the most important elements. Max pooling does this by only retaining the maximum value for each filtered area, and removing the remaining values.

![max_pooling](./images/max-pooling.png)

The image above is an example of max pooling with a 2x2 filter and stride of 2. The four 2x2 colors represent each time the filter was applied to find the maximum value.

For example, [[1, 0], [4, 6]] becomes 6, because 6 is the maximum value in this set. Similarly, [[2, 3], [6, 8]] becomes 8.

TensorFlow provides the **`tf.nn.max_pool()`** function to apply max pooling to your convolutional layers.

**Pooling layer reduces the output size and prevent the network from overfitting by removing the less important features from the image thus reducing the # of parameters in future layers.**

In [4]:
conv_layer = tf.nn.max_pool(conv_layer, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

The `tf.nn.max_pool()` function performs max pooling with the ksize parameter as the size of the filter and the strides parameter as the length of the stride. 2x2 filters with a stride of 2x2 are common in practice.

The `ksize` and `strides` parameters are structured as 4-element lists, with each element corresponding to a dimension of the input tensor **`([batch, height, width, channels])`**. For both ksize and strides, the batch and channel dimensions are typically set to 1

Recently, pooling layers have fallen out of favor. Some reasons are:

- Recent datasets are so big and complex we're more concerned about underfitting.
- Dropout is a much better regularizer.
- Pooling results in a loss of information. Think about the max pooling operation as an example. We only keep the largest of n numbers, thereby disregarding n-1 numbers completely.


In [5]:
input = tf.placeholder(tf.float32, (None, 4, 4, 5))
filter_shape = [1, 2, 2, 1]
strides = [1, 2, 2, 1]
padding = 'VALID'
pool = tf.nn.max_pool(input, filter_shape, strides, padding)

### Average pooling:
It is same as max pooling, the only difference is instead of taking max of the patch you take the average of it and output it.